In [48]:
import pandas as pd
import s2cell

# Load CSV into DataFrame

In [63]:
df_sample_csv=pd.read_csv('../00-data/sample/meta_shard_0.csv')
df_sample_csv['cellid']='_'
df_sample_csv['count']=1

In [60]:
df_sample_csv

,img,lat,lon,data,cellid,count
0,a6/c4/3038144879.jpg,48.9,2.3,ffd8ffe000104a46494600010100000100010000ffdb00...,_,1
1,a7/00/2633379373.jpg,45.9,0.2,ffd8ffe000104a46494600010100000100010000ffdb00...,_,1
2,03/2e/4493999629.jpg,50.9,6.9,ffd8ffe000104a46494600010100000100010000ffdb00...,_,1
3,c6/12/3073882997.jpg,51.5,7.5,ffd8ffe000104a46494600010100000100010000ffdb00...,_,1
4,0d/1d/3586372001.jpg,43.9,5.2,ffd8ffe000104a46494600010100000100010000ffdb00...,_,1
...,...,...,...,...,...,...
145,26/30/4883130713.jpg,51.5,-0.1,ffd8ffe000104a46494600010100000100010000ffdb00...,_,1
146,bd/f0/2340575216.jpg,48.9,2.3,ffd8ffe000104a46494600010100000100010000ffdb00...,_,1
147,98/b9/5657256708.jpg,46.3,-1.1,ffd8ffe000104a46494600010100000100010000ffdb00...,_,1
148,0d/89/8234978054.jpg,49.8,6.6,ffd8ffe000104a46494600010100000100010000ffdb00...,_,1


# Make geotagging using S2 and zoom

In [64]:
start_zoom = 3
end_zoom = 22
threshold = 5
def geohashing_zoom_s2(df_sample_csv,start_zoom,end_zoom,threshold):
    df_sample_csv[f'geohash_{start_zoom-1}'] = df_sample_csv.apply(lambda x: s2cell.lat_lon_to_cell_id(x.lat,x.lon,start_zoom-1),axis=1)
    df_sample_csv[f'geohash_{start_zoom}'] = df_sample_csv.apply(lambda x: s2cell.lat_lon_to_cell_id(x.lat,x.lon,start_zoom),axis=1)

    for zoom in range(start_zoom-1,end_zoom):
        if zoom > start_zoom-1 and (df_sample_csv.cellid=='_').sum()!=0:
            zoom_n1 = df_sample_csv[[f'geohash_{zoom-1}','count']]
            zoom_n1=zoom_n1.groupby(by=f'geohash_{zoom-1}').count().reset_index()
            zoom_n2 = df_sample_csv[[f'geohash_{zoom}','count']]
            zoom_n2=zoom_n2.groupby(by=f'geohash_{zoom}').count().reset_index()
            for i in range(len(df_sample_csv)):
                if df_sample_csv[f'geohash_{zoom}'][i] in list(zoom_n2[f'geohash_{zoom}'][zoom_n2['count']<threshold]):
                    for j in range(len(df_sample_csv)):
                        if df_sample_csv[f'geohash_{zoom-1}'][j]==df_sample_csv[f'geohash_{zoom-1}'][i]:
                            df_sample_csv.loc[j,'cellid']=df_sample_csv.loc[i,f'geohash_{zoom-1}']
            df_sample_csv.drop(columns=[f'geohash_{zoom-1}'],inplace=True)
            df_sample_csv[f'geohash_{zoom+1}'] = df_sample_csv.apply(lambda x: s2cell.lat_lon_to_cell_id(x.lat,x.lon,zoom+1),axis=1)
        else:
            next
    return df_sample_csv